In [2]:
import sys
# sys.path.append('/srv/share3/hagrawal9/project/sokoban/habitat-sim/')
# sys.path.append('/srv/share3/hagrawal9/project/sokoban/habitat-api/')

In [4]:
sys.path += []

['/coc/pskynet3/hagrawal9/project/sokoban/habitat-api',
 '/coc/pskynet3/hagrawal9/project/sokoban/habitat-sim',
 '/coc/pskynet3/hagrawal9/project/habitat/habitat-api/notebooks',
 '/srv/share3/hagrawal9/envs/habitat/lib/python36.zip',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/lib-dynload',
 '',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages',
 '/coc/pskynet3/hagrawal9/project/habitat/habitat-api',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/tensorflow-1.13.1-py3.6-linux-x86_64.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/tb_nightly-2.4.0a20200812-py3.6.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/moviepy-2.0.0.dev1-py3.6.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/wheel-0.34.2-py3.6.egg',
 '/srv/share3/hagrawal9/envs/habitat/lib/python3.6/site-packages/tensorflow_estimator-1.13.0-py3.6.egg',
 '/srv/share3/hagr

In [3]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

/coc/pskynet3/hagrawal9/project/habitat/habitat-api


In [4]:
import gzip
import habitat
import habitat_sim
import habitat_sim.bindings as hsim
import json
import matplotlib.pyplot as plt
import magnum as mn
import numpy as np
import os
import pandas as pd
import tqdm 

from habitat.config import Config
from examples.run_ortools import *
from habitat_sim.physics import MotionType
from habitat_sim.attributes import PhysicsObjectAttributes
from habitat_sim.helper import init_agent_state, make_cfg
from pathlib import Path
from PIL import Image
from typing import Any, Dict, List, Optional
from habitat.datasets.sokoban.shortest_path import create_data_model_for_object_goals
from habitat.utils.visualizations.maps import *

%matplotlib inline

In [5]:
%load_ext autoreload
%autoreload 2
from habitat.tasks.sokoban.sokoban_vis_utils import SokobanVisualization

In [ ]:
train_df = pd.read_pickle('data/sokoban_gibson_train.pkl')

In [ ]:
settings = {
    "max_frames": 10,
    "width": 256,  # Spatial resolution of the observations
    "height": 256,
    "scene": '',  # Scene path
    "default_agent": 0,
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": False,  # Semantic sensor
    "depth_sensor": True,  # Depth sensor
    "seed": 1,
    "enable_physics": True,
    "physics_config_file": "data/default.phys_scene_config.json", 
    "silent": False, 
    "num_objects": 10,
    "compute_shortest_path": False,
    "compute_action_shortest_path": False,
    "save_png": True,
    "render_to_ui": False
}

In [ ]:
navmesh_settings = habitat_sim.NavMeshSettings()
navmesh_settings.set_defaults()
navmesh_settings.agent_radius = 0.2
navmesh_settings.agent_height = 1.5
navmesh_settings.agent_max_climb = 0.1
# navmesh_settings.agent_max_slope = 0.1

In [ ]:
def get_rotation(sim, oid):
    quat = sim.get_rotation(oid)
    return np.array(quat.vector).tolist() + [quat.scalar]

In [ ]:
def geodesic_distance(sim, position_a, position_b):
    path = habitat_sim.MultiGoalShortestPath()
    path.requested_start = np.array(position_a, dtype=np.float32)
    if isinstance(position_b[0], List) or isinstance(
        position_b[0], np.ndarray
    ):
        path.requested_ends = np.array(position_b, dtype=np.float32)
    else:
        path.requested_ends = np.array(
            [np.array(position_b, dtype=np.float32)]
        )

    sim.pathfinder.find_path(path)

    return path.geodesic_distance

def euclidean_distance(position_a, position_b):
    return np.linalg.norm(
        np.array(position_b) - np.array(position_a), ord=2
    )

def validate_object(sim, agent_position, object_position, goal_position, object_positions, goal_positions, dist_threshold=15.0):
    ao_geo_dist = geodesic_distance(sim, agent_position, [object_position])
    ag_geo_dist = geodesic_distance(sim, agent_position, [goal_position])
    og_geo_dist = geodesic_distance(sim, object_position, [goal_position])
    
    ao_l2_dist = euclidean_distance(agent_position, object_position)
    ag_l2_dist = euclidean_distance(agent_position, goal_position)
    og_l2_dist = euclidean_distance(object_position, goal_position)
    
    ao_dist_ratio = ao_geo_dist / ao_l2_dist
    og_dist_ratio = og_geo_dist / og_l2_dist
    
    if ao_l2_dist < 1.0 or ao_geo_dist > 100 or np.abs(object_position[1] - agent_position[1]) > 0.2:
        # print("ao:", ao_l2_dist, ao_geo_dist)
        return False
    
    if ag_geo_dist > 100 or np.abs(goal_position[1] - agent_position[1]) > 0.2:
        # print("ag:", ag_geo_dist)
        return False
    
    if og_l2_dist < dist_threshold or og_geo_dist > 100 or np.abs(object_position[1] - goal_position[1]) > 0.2:
        # print("og:", og_l2_dist, og_geo_dist)
        return False
    
#     print("ao:", ao_l2_dist, ao_geo_dist)
#     print("ag:", ag_l2_dist, ag_geo_dist)
#     print("og:", og_l2_dist, og_geo_dist)

#     if ao_dist_ratio < 1.15 or og_dist_ratio < 1.15:
#         print(ao_geo_dist, ao_l2_dist, og_geo_dist, og_l2_dist)
#         return False
    
    for j, curr_pos in enumerate([object_position, goal_position]):
        for i, pos in enumerate(object_positions + goal_positions):
            geo_dist = geodesic_distance(sim, curr_pos, [pos])
            l2_dist = euclidean_distance(curr_pos, pos)
            
            if np.abs(curr_pos[1] - pos[1]) > 0.2:  # check height difference to assure s and
                # print("not on same floor", j, i)
                # are from same floor
                return False
            
            if sim.pathfinder.island_radius(curr_pos) != sim.pathfinder.island_radius(pos):
                # print("different island", j, i)
                return False 
            
            if l2_dist < 0.5 or geo_dist > 100:
            # if geo_dist > 100:
                # print("unreachable!", l2_dist, geo_dist, j, i)
                # print(l2_dist, j, i)
                return False

    return True
    
def validate_again(sim, object_positions, goal_positions):
    agent_position = np.array(sim.get_translation(0))
    
    for i, posi in enumerate([agent_position] + object_positions + goal_positions):
        for j, posj in enumerate(object_positions + goal_positions):
            if np.abs(posi[1] - posj[1]) > 0.2:  # check height difference to assure s and t are from the same floor
                print("diff floor", i, j, posi, posj)
                return False

            geo_dist = geodesic_distance(sim, posi, [posj])

            if geo_dist > 100:
                print(geo_dist, i, j)
                return False

    return True
    

In [ ]:
def init_test_scene_new(sim, navmesh_settings, num_objects, dist_thresh=5.0):
    #final_position
    final_position = sim.pathfinder.get_random_navigable_point()
    
    # add some objects in a grid
    object_lib_size = sim.get_physics_object_library_size()
    rand_obj_index = 0
    
    assert (
        object_lib_size > 0
    ), "!!!No objects loaded in library, aborting object instancing example!!!"

    # clear the objects if we are re-running this initializer
    for old_obj_id in sim.get_existing_object_ids()[1:]:
        sim.remove_object(old_obj_id)
    
    objectFile = 'data/test_assets/objects/sphere.glb'
    object_template = PhysicsObjectAttributes()
    object_template.set_render_mesh_handle(objectFile)
    object_template.set_bounding_box_collisions(True)
    object_template.set_scale(np.array([1.0, 1.0, 1.0]))
    object_template_id = sim._sim.load_object_template(object_template, "object")
    
    goal_objectFile = 'data/test_assets/objects/sphere_blue.glb'
    object_template = PhysicsObjectAttributes()
    object_template.set_render_mesh_handle(goal_objectFile)
    object_template.set_bounding_box_collisions(True)
    object_template.set_scale(np.array([1.0, 1.0, 1.0]))
    goal_template_id = sim._sim.load_object_template(object_template, "goal")
    
    object_positions = []
    goal_positions = []
    
    object_idxs = []
    goal_idxs = []
    
    agent_position = np.array(sim.get_translation(0))
    
    for obj_id in range(num_objects):
        count = 0

        object_id = sim.add_object(object_template_id)
        goal_id = sim.add_object(goal_template_id)
        
        while count < 100:
            while(True):
                object_position = sim.pathfinder.get_random_navigable_point()
                object_position[1] = agent_position[1]
                object_dist = sim.pathfinder.distance_to_closest_obstacle(object_position, max_search_radius=2.0)
                if object_dist > 0.5:
                    break
            
            while(True):
                goal_position = sim.pathfinder.get_random_navigable_point()
                goal_position[1] = agent_position[1]

                goal_dist = sim.pathfinder.distance_to_closest_obstacle(goal_position, max_search_radius=2.0)
            
                if goal_dist > 0.5:
                    break
            
            sim.set_object_motion_type(MotionType.DYNAMIC, object_id)
            sim.set_object_motion_type(MotionType.DYNAMIC, goal_id)
            
            # print("Object Dist: {}; Goal Dist: {}".format(object_dist, goal_dist))
            sim.set_translation(object_position, object_id)
            sim.set_translation(goal_position, goal_id)
            
            sim.set_object_motion_type(MotionType.STATIC, object_id)
            sim.set_object_motion_type(MotionType.STATIC, goal_id)
            
            sim.recompute_navmesh(
                sim.pathfinder, 
                navmesh_settings,
                include_static_objects=True
            )
            
            if validate_object(
                sim, agent_position, object_position, goal_position, 
                object_positions, goal_positions, 
                dist_threshold=dist_thresh
            ) and validate_again(sim, object_positions, goal_positions):
#                 print(
#                     "added object: "
#                     + str(object_id)
#                     + " at: "
#                     + str(object_position)
#                 )
                break
            
            count += 1
        
        if count < 100:
            object_positions.append(object_position)
            goal_positions.append(goal_position)    
            object_idxs.append(object_id)
            goal_idxs.append(goal_id)
            
            # print("Success in {}".format(count))
        else:
            sim.remove_object(object_id)
            sim.remove_object(goal_id)
            return object_positions, goal_positions, object_idxs, goal_idxs

        # recompute navmesh so that objects don't overlap with other existing objects. 
        sim.recompute_navmesh(
            sim.pathfinder, 
            navmesh_settings,
            include_static_objects=True
        )
        
    return object_positions, goal_positions, object_idxs, goal_idxs
        

In [ ]:
sim = None

In [ ]:
def init_episode_dict(sim, scene, episode_num):
    episode_dict = {
        'episode_id': episode_num, 
        'scene_id': scene,
        'start_position': np.array(sim.get_translation(0)).tolist(), 
        'start_rotation': get_rotation(sim, 0), 
        'info': {}, 
        'object_templates': [
            {
                'object_key': 'sphere_blue', 
                'object_template': 'data/test_assets/objects/sphere_blue.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_red', 
                'object_template': 'data/test_assets/objects/sphere.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_pink', 
                'object_template': 'data/test_assets/objects/sphere_pink.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_maroon', 
                'object_template': 'data/test_assets/objects/sphere_maroon.glb', 
                # other stats like scale etc that might be important. 
            },
            {
                'object_key': 'sphere_purple', 
                'object_template': 'data/test_assets/objects/sphere_purple.glb', 
                # other stats like scale etc that might be important. 
            }, 
            {
                'object_key': 'sphere_yellow', 
                'object_template': 'data/test_assets/objects/sphere_yellow.glb', 
                # other stats like scale etc that might be important. 
            },
            {
                'object_key': 'sphere_orange', 
                'object_template': 'data/test_assets/objects/sphere_orange.glb', 
                # other stats like scale etc that might be important. 
            },
            {
                'object_key': 'sphere_green', 
                'object_template': 'data/test_assets/objects/sphere_green.glb', 
                # other stats like scale etc that might be important. 
            }
        ],
        'objects': [

        ],
        'goals': [ 
            
        ], 
    }
    return episode_dict

def add_object_details(sim, episode_dict, num_objects, object_idxs):
    for i in range(num_objects):
        obj_id = object_idxs[i]
        object_template = {
            'object_id': i, 
            'object_key': i%8,
            'position': np.array(sim.get_translation(obj_id)).tolist(), 
            'rotation': get_rotation(sim, obj_id),
        }
        episode_dict['objects'].append(object_template)
    
    return episode_dict

def add_goal_details(sim, episode_dict, num_objects, goal_idxs):
    for i in range(num_objects):
        goal_id = goal_idxs[i]
        goal_template = {
            'position': np.array(sim.get_translation(goal_id)).tolist(), 
            'rotation': get_rotation(sim, goal_id),
        }
            
        episode_dict['goals'].append(goal_template)
    return episode_dict

In [ ]:
# set the number of objects to 1 always for now. 
def build_episode(episode_num, num_objects=1):
    episodes = {'episodes': []}
    for scene_id in train_df['id'].tolist():
        scene = 'data/scene_datasets/gibson_train_val/{}.glb'.format(scene_id)
        settings['scene'] = scene
        print(scene)
        
        cfg = make_cfg(settings)
        sim = habitat_sim.Simulator(cfg)
        sim.recompute_navmesh(sim.pathfinder, navmesh_settings, True)
        sv = SokobanVisualization(sim, navmesh_settings, map_resolution=(250, 250), num_samples=20000, draw_border=True)
    
        agent_id = settings["default_agent"]
        episode = 0
        dist_thresh = 3.0

        while episode < episode_num:
            start_state = init_agent_state(sim, navmesh_settings, agent_id)
            sim.recompute_navmesh(sim.pathfinder, navmesh_settings, True)

            num_object = np.random.choice(range(2, num_objects + 1))
            object_positions, goal_positions, object_idxs, goal_idxs = init_test_scene_new(
                sim, navmesh_settings, num_object, dist_thresh
            )
            
            sim.recompute_navmesh(sim.pathfinder, navmesh_settings, False)
            result = validate_again(sim, object_positions, goal_positions)
            if result == False or len(object_idxs) == 0:
                continue 
                
            num_object = len(object_idxs)
            
            assert len(object_idxs) == len(goal_idxs)
            episode_dict = init_episode_dict(sim, scene, episode)
            episode_dict = add_object_details(sim, episode_dict, num_object, object_idxs)
            episode_dict = add_goal_details(sim, episode_dict, num_object, goal_idxs)
            episodes['episodes'].append(episode_dict)
            print("\r Episode {} Object {}".format(episode, num_object), end=" ")
            episode += 1
            
        print("")
        sim.close()
    
    return episodes

In [ ]:
sim.close()

In [ ]:
scene_id = train_df['id'].tolist()[3]
scene = 'data/scene_datasets/gibson_train_val/{}.glb'.format(scene_id)
settings['scene'] = scene
print(scene)

cfg = make_cfg(settings)
agent_id = settings["default_agent"]
sim = habitat_sim.Simulator(cfg)
sv = SokobanVisualization(sim, navmesh_settings, map_resolution=(250, 250), num_samples=20000, draw_border=True)

In [ ]:
start_state = init_agent_state(sim, navmesh_settings, agent_id)

In [ ]:
tdmap, mp_list, rw_list = sv.get_topdown_map()
mp_list = np.array(mp_list)
rw_list = np.array(rw_list)

In [ ]:
from scipy.ndimage.measurements import label

In [ ]:
labelled, ncomp = label(tdmap)
print(ncomp)
indices = np.indices(tdmap.shape).T[:,:,[1, 0]]
comp_idxs = indices[labelled==2]
print(len(comp_idxs))
tdmap1 = np.array(tdmap)
tdmap1[comp_idxs[:, 0], comp_idxs[:, 1]] = 100
plt.imshow(tdmap1)
plt.show()

In [ ]:
# sample points from this, such that they are far apart
# they not too close to the wall
choices = np.random.choice(comp_idxs.shape[0], 2).astype(int)
mp_obj_choices = np.array(comp_idxs)[choices]
rw_obj_choices = [list(sv.from_grid(gpos[0], gpos[1])) for gpos in mp_obj_choices]

choices1 = np.random.choice(comp_idxs.shape[0], 2).astype(int)
mp_goal_choices = np.array(comp_idxs)[choices1]
rw_goal_choices = [list(sv.from_grid(gpos[0], gpos[1])) for gpos in mp_goal_choices]

print(rw_obj_choices[0])
objectFile = 'data/test_assets/objects/sphere.glb'
object_template = PhysicsObjectAttributes()
object_template.set_render_mesh_handle(objectFile)
object_template.set_bounding_box_collisions(True)
object_template.set_scale(np.array([1.0, 1.0, 1.0]))
object_template_id = sim._sim.load_object_template(object_template, "object")

goal_objectFile = 'data/test_assets/objects/sphere_blue.glb'
object_template = PhysicsObjectAttributes()
object_template.set_render_mesh_handle(goal_objectFile)
object_template.set_bounding_box_collisions(True)
object_template.set_scale(np.array([1.0, 1.0, 1.0]))
goal_template_id = sim._sim.load_object_template(object_template, "goal")


for i, choice in enumerate(mp_obj_choices):
    object_id = sim.add_object(object_template_id)
    goal_id = sim.add_object(goal_template_id)
        
    sim.set_object_motion_type(MotionType.DYNAMIC, object_id)
    sim.set_object_motion_type(MotionType.DYNAMIC, goal_id)

    # print("Object Dist: {}; Goal Dist: {}".format(object_dist, goal_dist))
    sim.set_translation(rw_obj_choices[i], object_id)
    sim.set_translation(rw_goal_choices[i], goal_id)

    sim.set_object_motion_type(MotionType.STATIC, object_id)
    sim.set_object_motion_type(MotionType.STATIC, goal_id)

    sim.recompute_navmesh(
        sim.pathfinder, 
        navmesh_settings,
        include_static_objects=True
    )
sim.pathfinder = sv.pf    
result = validate_again(sim, rw_obj_choices, rw_goal_choices)
print(result)
for obj_id in sim.get_existing_object_ids()[1:]:
    sim.remove_object(obj_id)

In [ ]:
start_state = init_agent_state(sim, navmesh_settings, agent_id)
object_positions, goal_positions, object_idxs, goal_idxs = init_test_scene_new(sim, navmesh_settings, 5)

In [ ]:
sim.pathfinder = sv.pf
validate_again(sim, object_positions, goal_positions)

In [ ]:
top_down_map = colorize_topdown_map(tdmap)

In [ ]:
top_down_map = sv.draw_topdown_map(rw_obj_choices, rw_goal_choices)

In [ ]:
plt.figure(figsize=(6,6), dpi=600)
plt.imshow(top_down_map)

In [ ]:
def geodesic_distance1(sim, pf, position_a, position_b):
    path = habitat_sim.MultiGoalShortestPath()
    path.requested_start = np.array(position_a, dtype=np.float32)
    if isinstance(position_b[0], List) or isinstance(
        position_b[0], np.ndarray
    ):
        path.requested_ends = np.array(position_b, dtype=np.float32)
    else:
        path.requested_ends = np.array(
            [np.array(position_b, dtype=np.float32)]
        )

    pf.find_path(path)

    return path.geodesic_distance

In [ ]:
validate_again(sim, object_positions, goal_positions)

In [ ]:
episode = 0
episodes = {'episodes': []}
episode_dict = init_episode_dict(sim, scene, episode)
episode_dict = add_object_details(sim, episode_dict, 5, object_idxs)
episode_dict = add_goal_details(sim, episode_dict, 5, goal_idxs)
episodes['episodes'].append(episode_dict)

In [ ]:
episodes['episodes'][0]['objects']

In [ ]:
plt.imshow(np.flip(tdmap1, axis=1).T)

In [ ]:
sim.close()

In [ ]:
episodes = build_episode(1000, num_objects=5)

data/scene_datasets/gibson_train_val/Fredericksburg.glb
 Episode 999 Object 4                                
data/scene_datasets/gibson_train_val/Hainesburg.glb
 Episode 407 Object 4    

In [18]:
with gzip.open('data/datasets/sokoban/coda/v1/train/content/object_5_scene_28_episode_1000_gibson.json.gz', "wt") as f:
    json.dump(episodes, f)

In [19]:
print("finished training episode")

finished training episode


In [23]:
train_df = pd.read_pickle('data/sokoban_gibson_val.pkl')
episodes = build_episode(50, num_objects=5)
with gzip.open('data/datasets/sokoban/coda/v1/train/content/object_5_scene_8_episode_50_gibson.json.gz', "wt") as f:
    json.dump(episodes, f)

data/scene_datasets/gibson_train_val/Kemblesville.glb
 Episode 49 Object 5 
data/scene_datasets/gibson_train_val/Sasakwa.glb
 Episode 49 Object 3  
data/scene_datasets/gibson_train_val/Chrisney.glb
 Episode 49 Object 4 
data/scene_datasets/gibson_train_val/Model.glb
 Episode 49 Object 4 
data/scene_datasets/gibson_train_val/Sodaville.glb
 Episode 49 Object 3 
data/scene_datasets/gibson_train_val/Castor.glb
 Episode 49 Object 5 
data/scene_datasets/gibson_train_val/Eagerville.glb
inf 0 0
 Episode 49 Object 1 


In [24]:
train_df = pd.read_pickle('data/sokoban_gibson_test.pkl')
episodes = build_episode(50, num_objects=5)
with gzip.open('data/datasets/sokoban/coda/v1/train/content/object_5_scene_9_gibson.json.gz', "wt") as f:
    json.dump(episodes, f)

data/scene_datasets/gibson_train_val/Stockertown.glb
 Episode 49 Object 3  
data/scene_datasets/gibson_train_val/Edgemere.glb
 Episode 49 Object 1 
data/scene_datasets/gibson_train_val/Wells.glb
 Episode 49 Object 2  
data/scene_datasets/gibson_train_val/Cokeville.glb
 Episode 49 Object 2 
data/scene_datasets/gibson_train_val/Howie.glb
 Episode 49 Object 2    
data/scene_datasets/gibson_train_val/Sands.glb
 Episode 49 Object 4 
data/scene_datasets/gibson_train_val/Sagerton.glb
 Episode 49 Object 4  
data/scene_datasets/gibson_train_val/Southfield.glb
 Episode 49 Object 2 
data/scene_datasets/gibson_train_val/Silerton.glb
 Episode 49 Object 4 


In [6]:
with gzip.open('data/datasets/sokoban/coda/v1/train/content/object_5_scene_28_episode_1000_gibson.json.gz', "rt") as f:
    ep = json.load(f)

In [9]:
import pandas as pd
epdf = pd.DataFrame.from_dict(ep['episodes'])

In [11]:
epdf['no_of_objects'] = epdf['objects'].apply(lambda x: len(x))

In [43]:
from collections import Counter
d = epdf.sort_values(by='no_of_objects')

In [44]:
del d['no_of_objects']
d = d.to_dict(orient='records')

In [45]:
ep['episodes'] = d

In [46]:
with gzip.open('data/datasets/sokoban/coda/v1/train/content/sorted_object_5_scene_28_episode_1000_gibson.json.gz', "wt") as f:
    json.dump(ep, f)

In [ ]:
obs = sim.get_sensor_observations()ascending=

In [ ]:
obs = sim.step("move_forward")

In [ ]:
obs = sim.step("turn_left")

In [ ]:
obs = sim.step("turn_right")

In [ ]:
obs = sim.step("grab_release")

In [ ]:
obs = sim.step("look_down")

In [ ]:
obs = sim.step("look_up")

In [ ]:
color_obs = obs["color_sensor"][:, :, :3]
color_obs[190:195, 125:130, :] = [0, 0, 255] 
depth_obs = obs["depth_sensor"]
depth_obs = ((depth_obs - np.min(depth_obs))/ np.max(depth_obs) * 255.0).astype(np.int)
depth_obs = np.transpose(np.stack([depth_obs]*3), (1,2,0))
plt.imshow(np.concatenate([color_obs, depth_obs], axis=1))

In [ ]:
import math 

from habitat_sim.utils.common import (
    d3_40_colors_rgb,
    download_and_unzip,
    quat_from_angle_axis,
)

def point_toward_object(sim, agent_id, obj_id):
    # obj_rot = sim.get_rotation(obj_id)
    # sim.set_rotation(obj_rot, agent_id)
    agent_pos = sim.get_translation(agent_id)
    obj_pos = sim.get_translation(obj_id)
    agent_to_obj = obj_pos - agent_pos
    agent_local_forward = np.array([0, 0, -1.0])
    flat_to_obj = np.array([agent_to_obj[0], 0.0, agent_to_obj[2]])
    det = (
            flat_to_obj[0] * agent_local_forward[2]
            - agent_local_forward[0] * flat_to_obj[2]
        )
    turn_angle = math.atan2(det, np.dot(agent_local_forward, flat_to_obj))
    quat = quat_from_angle_axis(turn_angle, np.array([0, 1.0, 0]))
    rot_quat = mn.Quaternion([quat.x, quat.y, quat.z], quat.w)
    sim.set_rotation(rot_quat, agent_id)

def move_towards_agent(sim, agent_id, obj_id):
    agent_pos = sim.get_translation(agent_id)
    obj_pos = sim.get_translation(obj_id)
    agent_to_obj = obj_pos - agent_pos
    agent_local_forward = np.array([0, 0, -1.0])
    flat_to_obj = np.array([agent_to_obj[0], 0.0, agent_to_obj[2]])
    sim.set_translation(obj_pos - (flat_to_obj * 1.0 / np.linalg.norm(flat_to_obj)), agent_id)

In [ ]:
def point_towards_goal(sim, agent_id, obj_id, final_id):
    obj_pos = sim.get_translation(obj_id)
    final_pos = sim.get_translation(final_id)
    final_to_obj = final_pos - obj_pos
    agent_local_forward = np.array([0, 0, -1.0])
    flat_to_obj = np.array([final_to_obj[0], 0.0, final_to_obj[2]])
    det = (
            flat_to_obj[0] * agent_local_forward[2]
            - agent_local_forward[0] * flat_to_obj[2]
        )
    turn_angle = math.atan2(det, np.dot(agent_local_forward, flat_to_obj))
    quat = quat_from_angle_axis(turn_angle, np.array([0, 1.0, 0]))
    rot_quat = mn.Quaternion([quat.x, quat.y, quat.z], quat.w)
    sim.set_rotation(rot_quat, agent_id)
    
def move_towards_goal(sim, agent_id, obj_id, final_id):
    agent_pos = sim.get_translation(agent_id)
    final_pos = sim.get_translation(final_id)
    obj_pos = sim.get_translation(obj_id)
    final_to_obj = final_pos - agent_pos
    agent_local_forward = np.array([0, 0, -1.0])
    flat_to_obj = np.array([final_to_obj[0], 0.0, final_to_obj[2]])
    sim.set_translation(final_pos - (flat_to_obj * 1.0 / np.linalg.norm(flat_to_obj)), agent_id)
    
def get_geodesic_distance_between_object_goal(sim, obj_id, final_id):
    final_pos = np.array(sim.get_translation(final_id))
    obj_pos = np.array(sim.get_translation(obj_id))
    dist = geodesic_distance(sim, obj_pos, [final_pos])
    print(dist)

In [ ]:
num_objects = len(episodes['episodes'][0]['objects'])
print(num_objects)

In [ ]:
obj_id = 1

In [ ]:
point_toward_object(sim, 0, obj_id)
move_towards_agent(sim, 0, obj_id)

In [ ]:
point_toward_object(sim, 0, 2*obj_id)
move_towards_agent(sim, 0, 2*obj_id)

In [ ]:
get_geodesic_distance_between_object_goal(sim, obj_id,  obj_id*2)

In [ ]:
get_geodesic_distance_between_object_goal(sim, 0,  obj_id)

In [ ]:
episodes

In [ ]:
len(episodes['episodes'])

In [ ]:
for episode in episodes['episodes']:
    print(len(episode['objects']))

In [ ]:
sim.pathfinder